In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# Matching dictionary for team names between events and tracking data
TEAM_NAMES = {
    "Canada": "Olympic (Women) - Canada",
    "USA": "Olympic (Women) - United States",
    "Finland": "Olympic (Women) - Finland",
    "ROC": "Olympic (Women) - Olympic Athletes from Russia",
    "Switzerland": "Olympic (Women) - Switzerland",
}

In [3]:
tracking_dir = "external/Big-Data-Cup-2022/data"
pbp_dir = "external/Big-Data-Cup-2022/data"
play_by_play_data_file = "pxp_womens_oly_2022_v2.csv"
power_play_info_file = "pp_info.csv"

In [4]:
play_by_play_data = pd.read_csv(os.path.join(tracking_dir, play_by_play_data_file))

In [5]:
def sort_row_by_col(row, col="player_name"):
    try:
        if row[f"{col}_2"] < row[col]:
            for col in ["player_name", "jn", "position"]:
                # for col in ["player_name"]:
                tmp = row[f"{col}_2"]
                row[f"{col}_2"] = row[col]
                row[col] = tmp
    except:
        print(row)

    return row

In [6]:
def run(game, game_date, team_name):
    this_game = play_by_play_data.copy()
    # power_play_info = pd.read_csv(os.path.join(pbp_dir, power_play_info_file))
    roster_info = pd.read_csv(os.path.join(tracking_dir, game, f"{game} roster.csv"), index_col=0)

    # Merge with roster data of current game
    this_game = this_game[this_game["game_date"] == game_date]
    this_game = this_game[this_game["team_name"] == team_name]
    this_game = this_game[(this_game["event"] == "Play") & this_game["event_successful"]]
    this_game = this_game.join(roster_info, on="player_name").join(roster_info, on="player_name_2", rsuffix="_2")
    # Exclude goalie
    # this_game = this_game[(this_game["position"] != "Goalie") & (this_game["position_2"] != "Goalie")]

    # By player
    # collected_passes = this_game.apply(lambda row: sort_row_by_col(row, col="player_name"), axis=1)
    # collected_passes = collected_passes.loc[:, ["player_name", "player_name_2"]].value_counts()
    # for period in [1, 2, 3]:
    # collected_passes = this_game[this_game["period"] == period]

    # By position
    collected_passes = this_game.apply(lambda row: sort_row_by_col(row, col="position"), axis=1)
    # Remove loops
    loops = collected_passes[collected_passes["position"] == collected_passes["position_2"]]
    print(f"#Loops: {len(loops)}")
    collected_passes = collected_passes.loc[:, ["position", "position_2"]].value_counts()
    passes_by_players = this_game.apply(lambda row: sort_row_by_col(row, col="player_name"), axis=1)
    passes_by_players.loc[:, ["position", "position_2"]].value_counts()
    print(collected_passes)

    # Write to CSV
    collected_passes.to_csv(f"{game} passes.csv")
    return loops, passes_by_players

In [7]:
game = "2022-02-14 Switzerland at Canada"
game_date = "14/2/2022"
team_name = TEAM_NAMES["Canada"]
loops, passes_by_players = run(game, game_date, team_name)

#Loops: 14
position       position_2   
Center         Right Wing       65
Left Wing      Right Wing       60
Left Defense   Left Wing        51
Right Defense  Right Wing       50
Center         Right Defense    44
Left Defense   Right Defense    42
               Right Wing       40
Center         Left Wing        38
               Left Defense     36
Left Wing      Right Defense    28
Goalie         Right Defense     6
Left Defense   Left Defense      6
Right Defense  Right Defense     4
Goalie         Left Defense      2
Left Wing      Left Wing         2
Center         Center            1
               Goalie            1
Right Wing     Right Wing        1
Name: count, dtype: int64


In [9]:
loops = loops.apply(lambda row: sort_row_by_col(row, col="player_name"), axis=1)
loops.loc[:, ["player_name", "position", "jn", "player_name_2", "jn_2"]].sort_values(["position", "jn", "jn_2"])

,player_name,position,jn,player_name_2,jn_2
6457,Marie-Philip Poulin,Center,29,Sarah Fillier,10
7073,Ella Shelton,Left Defense,17,Jocelyne Larocque,3
6294,Ashton Bell,Left Defense,21,Jocelyne Larocque,3
7062,Ashton Bell,Left Defense,21,Ella Shelton,17
7066,Ashton Bell,Left Defense,21,Ella Shelton,17
6425,Claire Thompson,Left Defense,42,Jocelyne Larocque,3
6705,Claire Thompson,Left Defense,42,Ella Shelton,17
6423,Melodie Daoust,Left Wing,15,Sarah Nurse,20
6438,Melodie Daoust,Left Wing,15,Sarah Nurse,20
6737,Jamie Lee Rattray,Right Defense,47,Renata Fast,14
